In [ ]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

In [ ]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [ ]:
pd.options.display.max_columns = 50
D='../input/m5-forecasting-accuracy/'

In [ ]:
h = 28 
max_lags = 70
tr_last = 1941
fday = datetime(2016,5, 23)

In [ ]:
state = 'TX' 
state_enc_dict={'CA': 0, 'TX' : 1, 'WI':2}

In [ ]:
def create_dt(is_train = True, nrows = None, first_day = 1200, state=state):
    prices = pd.read_csv(D+"sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(D+"calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv(D+"sales_train_evaluation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    dt = dt.query(f"state_id == {state_enc_dict[state]}")
    return dt

In [ ]:
def create_fea(dt):
    lags = [1, 2, 3, 5, 7]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 14]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [ ]:
FIRST_DAY = 1180 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [ ]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.tail()

In [ ]:
df.info()

In [ ]:
%%time

create_fea(df)
df.shape

In [ ]:
df.info()

In [ ]:
df.sample(10)

In [ ]:
# df.dropna(inplace = True)
# df.shape

In [ ]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id'] + ["event_name_1", "event_type_1",
                                                           "event_name_2", "event_type_2",]
snap_useless = {"CA": ["snap_TX", "snap_WI"], "TX": ["snap_CA", "snap_WI"], "WI": ["snap_TX", "snap_CA"] }
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", "state_id"] + snap_useless[state]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [ ]:
valid_days = ["d_" + str(_) for _ in range(1914, 1942)]
valid_inds = np.array([_ in valid_days for _ in df['d'].values])
valid_data = lgb.Dataset(X_train.iloc[valid_inds], label = y_train.iloc[valid_inds],categorical_feature=cat_feats,
                        free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !
# train_data = lgb.Dataset(X_train.iloc[~valid_inds], label = y_train.iloc[~valid_inds], categorical_feature=cat_feats, free_raw_data=False)
train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
valid_inds.sum()

In [ ]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.05,
        "lambda_l1" : 0.05,
#         "nthread" : 4
        'early_stopping_rounds': 50, 
        #'device' : 'gpu',
        'verbosity': 1,
        'num_iterations' : 10000,
        #'num_leaves': 128,
        #"min_data_in_leaf": 50,             
}

# SEED = 42
# params = {
#         'boosting_type': 'gbdt',         # Standart boosting type
#         'objective': 'regression',       # Standart loss for RMSE
#         'metric': ['rmse'],              # as we will use rmse as metric "proxy"
#         'subsample': 0.8,                
#         'subsample_freq': 1,
#         'learning_rate': 0.3,           # 0.5 is "fast enough" for us
#         'num_leaves': 2**7-1,            # We will need model only for fast check
#         'min_data_in_leaf': 2**6-1,      # So we want it to train faster even with drop in generalization 
#         'feature_fraction': 0.8,
# #         'n_estimators': 2000,            # We don't want to limit training (you can change 5000 to any big enough number)
#         'early_stopping_rounds': 30,     # We will stop training almost immediately (if it stops improving) 
#         'seed': SEED,
#         "lambda_l2" : 0.1,
#         'num_iterations' : 3000,
#         'verbose': -1,
#  }

In [ ]:
%%time
m_lgb = lgb.train(params, train_data, valid_sets = [train_data, valid_data], verbose_eval=100) 

In [ ]:
m_lgb.save_model(f"model_{state}.lgb")

In [ ]:
%%time

# alphas = [1.035, 1.03, 1.025, 1.02]
alphas = [1.0]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(icount, day.strftime("%Y-%m-%d"))
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(f"submission_{icount}.csv",index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)

In [ ]:
sub2 = sub.copy()
sub["id"] = sub["id"].str.replace("evaluation$", "validation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv(f"submission_{state}.csv",index=False)

In [ ]:
sub.id.nunique(), sub["id"].str.contains("evaluation$").sum()

In [ ]:
sub.shape